In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import RPi.GPIO as GPIO
import time
from datetime import datetime  as dt
import logging
import json

import sys



In [3]:
import logging
logger = logging.getLogger(__name__)

fmt = "[%(asctime)s][%(levelname)s] %(name)s :%(message)s"
logging.basicConfig(level=logging.DEBUG, format=fmt)


In [4]:

       
class ParticleCounter():

    def __init__(self,sampling_sec):
        self.sampling_sec = 30
        pass

    def set_pin_number(self, PIN=21):
        self.PIN = PIN
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.PIN, GPIO.IN)
        GPIO.setwarnings(False)
                
    def particle_count(self, count_times=10):
        self.count_times = count_times
        logging.info("Start")
        for i in range(self.count_times):
            logging.info("Rev:{0}".format(i))
            low_pulse_time_rst = self.get_low_pulse_time()
            concentration_rst = self.get_concentration(low_pulse_time_rst['during_low'],low_pulse_time_rst['t_end'])
            low_pulse_time_rst.update(concentration_rst)
            logger.info(" Rst:{0}".format(low_pulse_time_rst['concentration']))
            self.write_data(low_pulse_time_rst)
            
    def get_low_pulse_time(self):
        t_start = time.time()
        during_low,t_current = 0,0
        while True:
            during_low += self.time_low()
            t_current = (time.time() - t_start)
            if (t_current  > self.sampling_sec):
                break
        return {'during_low':during_low, 't_start':t_start, 't_end':t_current}
    
    def get_concentration(self,during_low_sec, sampling_sec):
        ratio = 100*during_low_sec/sampling_sec;
        concentration = 1.1*pow(ratio,3)-3.8*pow(ratio,2)+520*ratio+0.62        
        return {'ratio':ratio, 'concentration':concentration}
    
    def write_data(self,dict_obj):
        dict_obj['write_time'] = dt.now().strftime("%Y%m%d%H%M%s%Z")
        with open('file.txt', 'a') as file:
            file.write(json.dumps(dict_obj))
            file.write(',\n')


    def time_low(self):     
        t_start = time.time()
        t_end = time.time()

        if GPIO.input(self.PIN) == 1:
            return 0.0
        
        while GPIO.input(self.PIN) == 0:
            t_end = time.time()

        return (t_end - t_start)


In [5]:

# Make instance
device001 = ParticleCounter(30)

# Set signal pin
device001.set_pin_number(PIN=14)

# Excute
device001.particle_count(count_times=1000000)

# Finish
GPIO.cleanup()
print('==== Done ====')

[2021-09-05 11:28:55,124][INFO] root :Start
[2021-09-05 11:28:55,129][INFO] root :Rev:0
[2021-09-05 11:29:25,133][INFO] __main__ : Rst:{'during_low': 0.21079587936401367, 't_start': 1630808935.1334765, 't_end': 30.00000309944153, 'ratio': 0.7026528586189973, 'concentration': 364.5049524808377}
[2021-09-05 11:29:25,135][INFO] root :Rev:1
[2021-09-05 11:29:55,137][INFO] __main__ : Rst:{'during_low': 0.2104182243347168, 't_start': 1630808965.1374352, 't_end': 30.000001192092896, 'ratio': 0.7013940532448271, 'concentration': 363.85504261548175}
[2021-09-05 11:29:55,139][INFO] root :Rev:2
[2021-09-05 11:30:25,141][INFO] __main__ : Rst:{'during_low': 0.12263321876525879, 't_start': 1630808995.1412995, 't_end': 30.00000238418579, 'ratio': 0.40877736339748993, 'concentration': 212.62438990926114}
[2021-09-05 11:30:25,143][INFO] root :Rev:3
[2021-09-05 11:30:55,145][INFO] __main__ : Rst:{'during_low': 0.10862946510314941, 't_start': 1630809025.145232, 't_end': 30.00000262260437, 'ratio': 0.3620

KeyboardInterrupt: 